# Main Libraries for the Project

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
#Beth Computer Location File
#trainData = pd.read_csv("/Users/bethanydanner/Google_Drive/documents/python_code/clustering-based-anomaly-detection/Dataset/NSL-KDD/KDDTrain+.csv", header = None)
#testData = pd.read_csv("/Users/bethanydanner/Google_Drive/documents/python_code/clustering-based-anomaly-detection/Dataset/NSL-KDD/KDDTest+.csv", header = None)

# Reading the  Train Dataset and Checking if has missing Values

In [ ]:
#trainData = pd.read_csv("/Users/bethanydanner/Google_Drive/documents/python_code/clustering-based-anomaly-detection/Dataset/NSL-KDD/KDDTrain+.csv", header = None)
dataSet = pd.read_csv("/Users/jeremyperez/Jupyter/NSL-KDD/KDDTrain+.csv", header = None) 
#Run a Missing Value Ratio test to determine if any feature is missing values.
#If all ratios = 0.0, then data is not missing any values for any features.
dataSet.isnull().sum()/len(dataSet)*100


# Removing the attacks and Dificulty from the Data set
# For Categorical Test

In [ ]:
catg = dataSet.iloc[:,:-1].values # Get all the rows and all the clums except all the colums - 1
catgLabels = dataSet.iloc[:,42].values# Get all the rows and the colum number 42

# Removing all categorical data from the Data set
# For no Categorical Test

In [ ]:
noCatg = dataSet.iloc[:,[0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41]].values
noCatgLabels = dataSet.iloc[:,42].values 
################################################################################

# Encoding Categorical Data

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
#We use One hot encoding to pervent the machine learning to atribute the categorical data in order. 
#What one hot encoding(ColumnTransformer) does is, it takes a column which has categorical data, 
#which has been label encoded, and then splits the column into multiple columns.
#The numbers are replaced by 1s and 0s, depending on which column has what value
#We don't need to do a label encoded step because ColumnTransformer do one hot encode and label encode!

###############################################################################################################
#Manually encode
#protocols = {'tcp': 0,'udp': 1,'icmp': 2} 
#servers  = {'http': 0, 'domain_u': 1, 'sunrpc': 2, 'smtp': 3, 'ecr_i': 4, 'iso_tsap': 5, 'private': 6, 'finger': 7, 'ftp': 8, 'telnet': 9,'other': 10,'discard': 11, 'courier': 12, 'pop_3': 13, 'ldap': 14, 'eco_i': 15, 'ftp_data': 16, 'klogin': 17, 'auth': 18, 'mtp': 19, 'name': 20, 'netbios_ns': 21,'remote_job': 22,'supdup': 23,'uucp_path': 24,'Z39_50': 25,'csnet_ns': 26,'uucp': 27,'netbios_dgm': 28,'urp_i': 29,'domain': 30,'bgp':31,'gopher': 32,'vmnet': 33,'systat': 34,'http_443': 35,'efs': 36,'whois': 37,'imap4': 38,'echo': 39,'link': 40,'login': 41,'kshell': 42,'sql_net': 43,'time': 44,'hostnames': 45,'exec': 46,'ntp_u': 47,'nntp': 48,'ctf': 49,'ssh': 50,'daytime': 51,'shell': 52,'netstat': 53,'nnsp': 54,'IRC': 55,'pop_2': 56,'printer': 57,'tim_i': 58,'pm_dump': 59,'red_i': 60,'netbios_ssn': 61,'rje': 62,'X11': 63,'urh_i': 64,'http_8001': 65,'aol': 66,'http_2784': 67,'tftp_u': 68,'harvest': 69} 
#servers_error  = {'REJ': 0, 'SF': 1, 'S0': 2, 'RSTR': 3, 'RSTO': 4,'SH': 5,'S1': 6,'RSTOS0': 7,'S3': 8,'S2': 9,'OTH': 10} 
#attacks  = {'normal': "normal", 'neptune': "DoS", 'warezclient': "R2L", 'ipsweep': "Probe", 'mscan': 4, 'back': "DoS", 'smurf': "DoS", 'mailbomb': 7, 'apache2': 8, 'rootkit': "U2R",'satan': "Probe", 'processtable': 12, 'guess_passwd': "R2L", 'saint': 14,'portsweep': "Probe",'teardrop': "DoS",'nmap': "Probe",'pod': "DoS",'ftp_write': "R2L",'multihop': "R2L",'buffer_overflow': "U2R",'imap': "R2L",'warezmaster': "R2L",'phf': "R2L",'land': "DoS",'loadmodule': "U2R",'spy': "R2L",'perl': "U2R",'snmpgetattack': 27,'httptunnel': 28,'ps': 29,'snmpguess': 30,'named': 31,'sendmail':32,'xterm':33,'worm': 34,'xlock': 35,'xsnoop': 36,'sqlattack': 37,'udpstorm':38} 

#Y[1] = [attacks[item] for item in Y[1]]
#Y[2] = [servers[item] for item in Y[2]]
#Y[3] = [servers_error[item] for item in Y[3]]
#Y[42] = [attacks[item] for item in Y[42]]
###############################################################################################################

#One Hot Encoding to the attacks
#Encoding the Independient Variable
transformX = ColumnTransformer([("Servers", OneHotEncoder(categories = "auto"), [1,2,3])], remainder="passthrough")
X = transformX.fit_transform(X)

# Normalizing the data

In [ ]:
#Because we are using numerical-value-only clustering techniques to analyze the NSL-KDD dataset,
#we need to normalize the values in the dataset, as Ibrahim., et. al. describe (page 112).
#We complete the normalization process below:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer().fit(X)
X = normalizer.transform(X)

# Elbow Method 

In [ ]:
#Elbow method to find the best number of culster
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++',max_iter = 300,n_init = 10,random_state = 0)
    kmeans.fit(catg)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# KMeans

In [ ]:
#Applying K-mea(n_clusters = 5)
KMEANS = KMeans(n_clusters = 4, init = 'k-means++',max_iter = 300,n_init = 10,random_state = 0)
kmeans = KMEANS.fit() # Variable for categoricaal or no categorical
kmeans.labels_

# Analyzing Results of K-means by Crosstab

In [ ]:
pd.crosstab(,kmeans.labels_)

# Analyzing Results by Graph

In [ ]:
#Visual representation of the clusters
#plt.scatter(X[y_kmeans ==0,0],X[y_kmeans == 0,1], s = 21, c = 'red', label = 'cluster1')
#plt.scatter(X[y_kmeans ==1,0],X[y_kmeans == 1,1], s = 21, c = 'yellow', label = 'cluster2')
#plt.scatter(X[y_kmeans ==2,0],X[y_kmeans == 2,1], s = 21, c = 'cyan', label = 'cluster3')
#plt.scatter(X[y_kmeans ==3,0],X[y_kmeans == 3,1], s = 21, c = 'orange', label = 'cluster4')
#plt.scatter(X[y_kmeans ==4,0],X[y_kmeans == 4,1], s = 21, c = 'black', label = 'cluster5')
#plt.scatter(kmeans.cluster_centers_[:, 0],kmeans.cluster_centers_[:, 1],s = 300, c = 'purple', label = 'Centroids')
#plt.title('Clusters of Attacks')
#plt.xlabel('Numbers of Attacks')
#plt.ylabel('Types of Attacks')
#plt.legend()
#plt.show()

# DBSCAN 

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score
#OPTICS
# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit()
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

# Analyzing Results of DBSCAN by Crosstab

In [ ]:
pd.crosstab(,labels)

# Encoding  the Data for F-score

In [ ]:
attackEncodingCluster  = {'U2R': 1 ,'Probe':1 ,'R2L': 1,'DoS': 1,'normal':0}
attacks[:] = [attackEncodingCluster[item] for item in attacks[:]]
attackEncodingCluster = np.array(attacks,dtype = int

# F-Score implementation

In [ ]:
from sklearn.metrics import f1_score
attackEncodingCluster  = {'U2R': 1 ,'Probe':1 ,'R2L': 1,'DoS': 1,'normal':0}
attacks[:] = [attackEncodingCluster[item] for item in attacks[:]]
attackEncodingCluster = np.array(attacks,dtype = int)
f1 = f1_score(Y,kmeans.labels_, average="weighted") #[None, 'micro', 'macro', 'weighted']
f1